In [28]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import itertools
from scipy import stats
import warnings
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
!pip install xlsxwriter
pd.options.mode.chained_assignment = None  # default='warn'
import statsmodels.api as sm
from statsmodels.formula.api import ols
## https://www.kaggle.com/code/alexmaszanski/two-way-anova-with-python/notebook
from scipy.stats import linregress
import glob
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import re
from statsmodels.stats.anova import AnovaRM
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import pingouin as pg


**There are four assumptions that must be met before using two-way ANOVA:**

- Normality: Observations from the sample population are normally distributed.
- Sample Size: The number of observations must be the same for each group.
- Equal Variances: The variances for each group are equal.
- Independence: Observations in each group are independent.

In [29]:
def custom_sort(item):
    if item.endswith('basal'):
        return 0
    else:
        return int(item.split('uM')[0].split('_')[-1])

In [30]:
currdir= os.getcwd()
parent = os.path.dirname(currdir)
gparent=os.path.dirname(parent)
lines_to_skip = 10 # adjust 

# count the number of header lines
header_lines = 3 # adjust
EXPERIMENT_TIME = "1_MIN"
plate_type = 'DIV19' #adjust

PERTURBATION = "CNO" # adjust
firstTableHeading = "Number of Spikes"
## conditions taken: ### we will define this as dose1,dose2, dose3, dose4, each incrementing in terms of doses

FOLDER_PATH = f"{parent}/data/Div19_CNO/Div19_1minute" #
folder_dict = {}
balances= {"basal": "",
        "dose1": "",
        "dose2": "",
        "dose3": "",
        "dose4": ""
        }

## WALK Through folder of interest
for dirpath, dirnames, filenames in os.walk(FOLDER_PATH):
    if dirpath != FOLDER_PATH:
        folder_dict[dirpath.split("/")[-1]] = dirpath
    elif not dirnames:
        folder_dict['basal'] = dirpath

## get doses names in increasing order 
dose_names = [name for name in folder_dict.keys() if 'basal' not in name]
dose_names.sort(key=custom_sort)

# Initialize balances dictionaryd
balances = {"basal": "basal"}

## map sorted doses
for i, dose_name in enumerate(dose_names, start=1):
    balances[f"dose{i}"] = dose_name.split("_")[-1]

## repeated measures ANOVA test?
ANOVARM = True

print("Balances:", balances)
folder_dict

Balances: {'basal': 'basal', 'dose1': '5uMCNO', 'dose2': '10uMCNO', 'dose3': '15uMCNO', 'dose4': '20uMCNO'}


{'Div19_5uMCNO': '/home/poojaparameswaran/Documents/SoderlingLab/MultipleElectrodeAnalysisANOVA/data/Div19_CNO/Div19_1minute/Div19_5uMCNO',
 'Div19_10uMCNO': '/home/poojaparameswaran/Documents/SoderlingLab/MultipleElectrodeAnalysisANOVA/data/Div19_CNO/Div19_1minute/Div19_10uMCNO',
 'Div19_basal': '/home/poojaparameswaran/Documents/SoderlingLab/MultipleElectrodeAnalysisANOVA/data/Div19_CNO/Div19_1minute/Div19_basal',
 'Div19_20uMCNO': '/home/poojaparameswaran/Documents/SoderlingLab/MultipleElectrodeAnalysisANOVA/data/Div19_CNO/Div19_1minute/Div19_20uMCNO',
 'Div19_15uMCNO': '/home/poojaparameswaran/Documents/SoderlingLab/MultipleElectrodeAnalysisANOVA/data/Div19_CNO/Div19_1minute/Div19_15uMCNO'}

In [31]:
## now only select the control and maximum
balances = {k: balances[k] for k in ('basal', 'dose4')}
getnums_f = lambda x: [int(num) for num in re.findall(r'\d+', x.split("_")[-1])]
concnums = list(map(getnums_f, list(folder_dict.keys())))
concnums = list(itertools.chain(*concnums))
folder_dict = {k: folder_dict[k] for k in folder_dict.keys() if any(x in k for x in ('basal', str(max(concnums))))}
folder_dict, balances

({'Div19_basal': '/home/poojaparameswaran/Documents/SoderlingLab/MultipleElectrodeAnalysisANOVA/data/Div19_CNO/Div19_1minute/Div19_basal',
  'Div19_20uMCNO': '/home/poojaparameswaran/Documents/SoderlingLab/MultipleElectrodeAnalysisANOVA/data/Div19_CNO/Div19_1minute/Div19_20uMCNO'},
 {'basal': 'basal', 'dose4': '20uMCNO'})

In [32]:
def read_csvs_to_dict(folder, lines_to_skip=10):
    files = glob.glob(os.path.join(folder, "*.csv"))
    data_dict = {}
    for file in files:
        # Extracting the plate number from the filename
        cond = (file.split("/")[-2].split("_")[-1])
        plate_num = re.search(r'plate\s?(\d+)', file, re.IGNORECASE)
        if plate_num:
            plate_key = f"plate_{plate_num.group(1)}"
            df = pd.read_csv(file, skiprows = lines_to_skip)
            df = df.drop(columns=['Unnamed: 9'])
            df['Subject'] = plate_key
            data_dict[plate_key] = df
    return data_dict
all_files_dict = {}

for dose, folder in folder_dict.items():
    all_files_dict[dose] = read_csvs_to_dict(folder)
    
for dose, files_dict in all_files_dict.items():
    print(files_dict.keys())

dict_keys(['plate_1', 'plate_3'])
dict_keys(['plate_3', 'plate_1'])


In [33]:
def concat_plates(plates_dict, firstHeading):
    experiment = {}
    testparams_concat = {}
    for plate, df in plates_dict.items():
        maxrows = df.shape[0]
        vals_between_tables = 16
        titles = [heading_ind for heading_ind in range(14, maxrows+16, 16)]
        titles.insert(0,0)
        # # Empty dictionary to store dataframes
        # Loop through start indices
        for ind in range(len(titles) -1):
            if ind == 0:
                ## Handle mean firing rate
                key = firstHeading
                table = df.iloc[titles[ind] : titles[ind +1], :]
                # table = rename_row_names(table)
            elif ind != 0:
                table = df.iloc[titles[ind] : titles[ind +1], :]
                key = table.iloc[0][0] # just get name
                table = df.iloc[titles[ind] +2: titles[ind +1], :]

            table.set_index('Unnamed: 0', inplace=True)
            if key not in experiment:
                experiment[key] = []
            experiment[key].append(table)
    
    for testparam, df_list in experiment.items():
        concated_table = pd.concat(df_list, ignore_index=False)
        testparams_concat[testparam] = concated_table
    
    testparams_concat = {key.strip(): value for key, value in testparams_concat.items()}
    return testparams_concat
doses = {}
for dose, plates in all_files_dict.items():
    doses[dose] = concat_plates(plates, firstTableHeading)

doses.keys()

dict_keys(['Div19_basal', 'Div19_20uMCNO'])

In [34]:
def remove_mean_SEM(test_dict, chemical_type):
    for key, value in test_dict.items():
        to_drop = [index for index in ['B Mean', 'B SEM'] if index in value.index]
        if to_drop:
            test_dict[key] = value.drop(index=to_drop)
        test_dict[key]['Chemical_Type']=chemical_type
    return test_dict
for dose, parameter_dict in doses.items():
    doses[dose] = remove_mean_SEM(parameter_dict, dose)

doses.keys()

dict_keys(['Div19_basal', 'Div19_20uMCNO'])

In [35]:
def rename_row_names(experiment_df):
    # 'Unnamed: 0' is a column
    experiment_df = experiment_df.reset_index()
    experiment_df = experiment_df.drop(columns="Unnamed: 0", errors="ignore")
    plates = list(experiment_df['Subject'])
    chems = list(experiment_df['Chemical_Type'].apply(lambda x: x.split("_")[-1]))
    experiment_df.index = [f"Replicate{i+1}" for i in range(len(experiment_df))]
    experiment_df.drop(columns='Subject', inplace=True)
    return experiment_df
for dose, test_dict in doses.items():
    for test_name, df in test_dict.items():
        test_dict[test_name] = rename_row_names(df)
    doses[dose] = test_dict


In [36]:
doses['Div19_20uMCNO']['Number of Spikes']

,GPR37L 1,GPR37L 2,LGI2 A12,SLITRK5,THSD7 1,THSD7 2,unt1,unt2,Chemical_Type
Replicate1,1932.000000,2826.000000,2486.000000,81.000000,1796.000000,1455.000000,509.000000,2431.000000,Div19_20uMCNO
Replicate2,817.000000,995.000000,1553.000000,1252.000000,893.000000,1374.000000,1238.000000,988.000000,Div19_20uMCNO
Replicate3,957.000000,1926.000000,2348.000000,1204.000000,1290.000000,388.000000,2803.000000,1645.000000,Div19_20uMCNO
Replicate4,1101.000000,1207.000000,1075.000000,2540.000000,1220.000000,500.000000,2896.000000,1109.000000,Div19_20uMCNO
Replicate5,1509.000000,3325.000000,2611.000000,2128.000000,1082.000000,412.000000,1805.000000,518.000000,Div19_20uMCNO
Replicate6,713.000000,1270.000000,1975.000000,1782.000000,1587.000000,1361.000000,1186.000000,2419.000000,Div19_20uMCNO
Replicate7,2032.000000,2624.000000,2674.000000,2912.000000,1156.000000,1865.000000,3912.000000,1507.000000,Div19_20uMCNO
Replicate8,3018.000000,2727.000000,3000.000000,2117.000000,2299.000000,1389.000000,942.000000,1274.000000,Div19_20uMCNO
Replicate9,2287.000000,5105.000000,2144.000000,2642.000000,2288.000000,3114.000000,3339.000000,3987.000000,Div19_20uMCNO
Replicate10,4001.000000,1849.000000,2628.000000,1570.000000,3449.000000,2569.000000,2522.000000,2967.000000,Div19_20uMCNO


In [37]:
balances.keys(), balances.values(), doses.keys()

(dict_keys(['basal', 'dose4']),
 dict_values(['basal', '20uMCNO']),
 dict_keys(['Div19_basal', 'Div19_20uMCNO']))

In [38]:
dmapped = {}
for bkey, bval in balances.items():
    for dkey, dval in doses.items():
        if bval in dkey:
            dmapped[bkey] = dval

dmapped.keys(), type(list(dmapped.values())[0])

(dict_keys(['basal', 'dose4']), dict)

In [39]:
def combine_chem_types(dict_of_alldicts):
    all_concat = {key: None for key in dict_of_alldicts['basal']}
    for type in dict_of_alldicts.keys():
        for test_parameter, df in dict_of_alldicts[type].items():
            all_concat[test_parameter]=pd.concat([all_concat[test_parameter], df], ignore_index=False)
    for testparam, df in all_concat.items():
        df.columns = df.columns.str.replace(' ', '_')
        for gene in df.columns:
            if gene != "Chemical_Type" and gene !="Subject":
                df[gene]= df[gene].astype(float)
    return all_concat

tests_mapped = combine_chem_types(dmapped)
print(tests_mapped['Network Burst Duration - Avg (sec)']['Chemical_Type'].unique())
tests_mapped['Burst Duration - Avg (sec)']

['Div19_basal' 'Div19_20uMCNO']


,GPR37L_1,GPR37L_2,LGI2_A12,SLITRK5,THSD7_1,THSD7_2,unt1,unt2,Chemical_Type
Replicate1,1.9e-01,1.9e-01,1.4e-01,9.2e-02,1.2e-01,1.3e-01,2.3e-01,1.3e-01,Div19_basal
Replicate2,1.0e-01,1.4e-01,NaN,2.2e-01,1.2e-01,NaN,1.8e-01,1.4e-01,Div19_basal
Replicate3,1.1e-01,1.5e-01,2.7e-01,1.4e-01,NaN,NaN,2.2e-01,NaN,Div19_basal
Replicate4,1.3e-01,1.7e-01,1.4e-01,1.1e-01,1.0e-01,8.3e-02,1.4e-01,2.2e-01,Div19_basal
Replicate5,9.5e-02,1.6e-01,8.2e-02,2.5e-01,NaN,NaN,1.3e-01,1.3e-01,Div19_basal
Replicate6,1.3e-01,4.5e-01,4.3e-01,1.8e-01,NaN,1.3e-01,1.4e-01,1.3e-01,Div19_basal
Replicate7,1.6e-01,1.5e-01,NaN,1.3e-01,1.7e-01,2.5e-01,1.6e-01,1.4e-01,Div19_basal
Replicate8,1.1e-01,1.8e-01,NaN,5.2e-01,NaN,9.4e-02,1.4e-01,NaN,Div19_basal
Replicate9,1.2e-01,1.5e-01,NaN,1.8e-01,NaN,1.4e-01,1.5e-01,1.1e-01,Div19_basal
Replicate10,1.1e-01,1.5e-01,2.2e-01,1.6e-01,2.2e-01,1.0e-01,1.4e-01,NaN,Div19_basal


In [40]:
def impute_nans(testparam, table):
    chem_types = table['Chemical_Type']
    table = table.drop(columns=['Chemical_Type'])
    original_index = table.index
    imputer = IterativeImputer(max_iter=100, random_state=0)
    imputed_data = imputer.fit_transform(table)
    # Convert the imputed data back to a DataFrame
    imputed_df = pd.DataFrame(imputed_data, columns=table.columns, index=original_index)
    imputed_df['Chemical_Type'] = chem_types
    return imputed_df

# if nans in all columns, skip. need to remove chemtype and subject bc these will never have nans
testsI_mapped = {test_param: impute_nans(test_param, table) for test_param, table in tests_mapped.items() \
    if not table.loc[:, ~(table.columns.isin(['Chemical_Type']))].isna().all().all()
} 
testsI_mapped['Burst Duration - Avg (sec)']

,GPR37L_1,GPR37L_2,LGI2_A12,SLITRK5,THSD7_1,THSD7_2,unt1,unt2,Chemical_Type
Replicate1,1.9e-01,1.9e-01,1.4e-01,9.2e-02,1.2e-01,1.3e-01,2.3e-01,1.3e-01,Div19_basal
Replicate2,1.0e-01,1.4e-01,1.3e-01,2.2e-01,1.2e-01,1.7e-01,1.8e-01,1.4e-01,Div19_basal
Replicate3,1.1e-01,1.5e-01,2.7e-01,1.4e-01,1.9e-01,2.0e-01,2.2e-01,2.5e-01,Div19_basal
Replicate4,1.3e-01,1.7e-01,1.4e-01,1.1e-01,1.0e-01,8.3e-02,1.4e-01,2.2e-01,Div19_basal
Replicate5,9.5e-02,1.6e-01,8.2e-02,2.5e-01,1.6e-01,1.8e-01,1.3e-01,1.3e-01,Div19_basal
Replicate6,1.3e-01,4.5e-01,4.3e-01,1.8e-01,2.5e-01,1.3e-01,1.4e-01,1.3e-01,Div19_basal
Replicate7,1.6e-01,1.5e-01,1.5e-01,1.3e-01,1.7e-01,2.5e-01,1.6e-01,1.4e-01,Div19_basal
Replicate8,1.1e-01,1.8e-01,2.6e-01,5.2e-01,1.9e-01,9.4e-02,1.4e-01,3.1e-01,Div19_basal
Replicate9,1.2e-01,1.5e-01,1.4e-01,1.8e-01,1.7e-01,1.4e-01,1.5e-01,1.1e-01,Div19_basal
Replicate10,1.1e-01,1.5e-01,2.2e-01,1.6e-01,2.2e-01,1.0e-01,1.4e-01,2.0e-01,Div19_basal


In [41]:
def melt_df_Long(df):
    df = df.reset_index().rename(columns={'index': 'Subject'})
    new_df = pd.melt(df, id_vars=['Subject','Chemical_Type'], var_name='Gene', value_name='Quantity')

    return new_df

# Apply the function to your data
testsI_long = {test_param: melt_df_Long(table) for test_param, table in testsI_mapped.items()}


In [42]:
testsI_long['Number of Active Electrodes']

,Subject,Chemical_Type,Gene,Quantity
0,Replicate1,Div19_basal,GPR37L_1,8.0e+00
1,Replicate2,Div19_basal,GPR37L_1,8.0e+00
2,Replicate3,Div19_basal,GPR37L_1,8.0e+00
3,Replicate4,Div19_basal,GPR37L_1,8.0e+00
4,Replicate5,Div19_basal,GPR37L_1,8.0e+00
...,...,...,...,...
379,Replicate20,Div19_20uMCNO,unt2,8.0e+00
380,Replicate21,Div19_20uMCNO,unt2,8.0e+00
381,Replicate22,Div19_20uMCNO,unt2,8.0e+00
382,Replicate23,Div19_20uMCNO,unt2,8.0e+00


In [43]:
## REMOVE UNT1
for testparam, table in testsI_long.items():
    testsI_long[testparam] = table[table['Gene'] != 'unt1']
testsI_long['Number of Active Electrodes']

,Subject,Chemical_Type,Gene,Quantity
0,Replicate1,Div19_basal,GPR37L_1,8.0e+00
1,Replicate2,Div19_basal,GPR37L_1,8.0e+00
2,Replicate3,Div19_basal,GPR37L_1,8.0e+00
3,Replicate4,Div19_basal,GPR37L_1,8.0e+00
4,Replicate5,Div19_basal,GPR37L_1,8.0e+00
...,...,...,...,...
379,Replicate20,Div19_20uMCNO,unt2,8.0e+00
380,Replicate21,Div19_20uMCNO,unt2,8.0e+00
381,Replicate22,Div19_20uMCNO,unt2,8.0e+00
382,Replicate23,Div19_20uMCNO,unt2,8.0e+00


In [44]:
def categorize_gene_vs_control(testparam_df, control_var='unt2'):
    genes = list(testparam_df['Gene'].unique())
    mutants = [gene for gene in genes if "unt" not in gene]
    mutVcont_dict= {mut: pd.DataFrame for mut in mutants}
    for mutant in mutants:
        tiny_df = testparam_df[(testparam_df['Gene'] == control_var) |\
                   (testparam_df['Gene'] == mutant)]
        mutVcont_dict[mutant] = tiny_df
    return mutVcont_dict

## created from testsI_mapped
comparisons_long = {}
for test_param, table in testsI_long.items():
    comparisons_long[test_param] = categorize_gene_vs_control(table, control_var='unt2')
comparisons_long['Number of Active Electrodes']['GPR37L_1']

,Subject,Chemical_Type,Gene,Quantity
0,Replicate1,Div19_basal,GPR37L_1,8.0e+00
1,Replicate2,Div19_basal,GPR37L_1,8.0e+00
2,Replicate3,Div19_basal,GPR37L_1,8.0e+00
3,Replicate4,Div19_basal,GPR37L_1,8.0e+00
4,Replicate5,Div19_basal,GPR37L_1,8.0e+00
...,...,...,...,...
379,Replicate20,Div19_20uMCNO,unt2,8.0e+00
380,Replicate21,Div19_20uMCNO,unt2,8.0e+00
381,Replicate22,Div19_20uMCNO,unt2,8.0e+00
382,Replicate23,Div19_20uMCNO,unt2,8.0e+00


In [45]:
comparisons_long['Number of Active Electrodes']['GPR37L_1']['Subject'].unique()

array(['Replicate1', 'Replicate2', 'Replicate3', 'Replicate4',
       'Replicate5', 'Replicate6', 'Replicate7', 'Replicate8',
       'Replicate9', 'Replicate10', 'Replicate11', 'Replicate12',
       'Replicate13', 'Replicate14', 'Replicate15', 'Replicate16',
       'Replicate17', 'Replicate18', 'Replicate19', 'Replicate20',
       'Replicate21', 'Replicate22', 'Replicate23', 'Replicate24'],
      dtype=object)

In [46]:
### You want to troubleshoot w PRISM??

TESTPARAM = 'Area Under Normalized Cross-Correlation'
GENEINT = 'GPR37L_1'
# TestParamsSectioned_dict[TESTPARAM][GENEINT].to_csv(f"{parent}/data/PRISM_inputs/{TESTPARAM}_{GENEINT}.csv")

In [47]:
comparisons_long['Number of Active Electrodes']['GPR37L_1']

,Subject,Chemical_Type,Gene,Quantity
0,Replicate1,Div19_basal,GPR37L_1,8.0e+00
1,Replicate2,Div19_basal,GPR37L_1,8.0e+00
2,Replicate3,Div19_basal,GPR37L_1,8.0e+00
3,Replicate4,Div19_basal,GPR37L_1,8.0e+00
4,Replicate5,Div19_basal,GPR37L_1,8.0e+00
...,...,...,...,...
379,Replicate20,Div19_20uMCNO,unt2,8.0e+00
380,Replicate21,Div19_20uMCNO,unt2,8.0e+00
381,Replicate22,Div19_20uMCNO,unt2,8.0e+00
382,Replicate23,Div19_20uMCNO,unt2,8.0e+00


In [48]:
def ensure_dirs_exists(path):
    directory = os.path.dirname(path)  # Get the directory part of the file path
    if not os.path.exists(directory):
        os.makedirs(directory)
    return

In [49]:
def calculate_ANOVA(testparameter, df, anovaRM=True,
                    control_var="unt2"):
    """_summary_
    In a two-way repeated anova measurement test, 1 factor== within-subject, each 
        subject is exposed to all levels of this factor (chemical_Type).
        -Value is dependent variable
        -Gene is index, each gene is eitehr control or mutation so it is key identifier
        -Chemical_type is change within each subject
    
    Args:
        testparameter (_type_): _description_from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

    Returns:
        Anova TABLE: showing mut (1) vs control (1), significance of gene
                        perturbation, gene&chemical
            'Source': Name of the within-group factor
        if RM:
            'ddof1': Degrees of freedom (numerator)

            'ddof2': Degrees of freedom (denominator)

            'F': F-value ## if null hypothesis is true, f to have val close to 1
                - variation between sample means/variation within the samples.

            'p-unc': Uncorrected p-value

            'ng2': Generalized eta-square effect size 
            - to understand magnitude of these differences. 
             - 0.02, 2.26% of variance in quanitty is caused by that.
             - small value indicates no individual factors have a large impact on
             dependent variable.

            'eps': Greenhouse-Geisser epsilon factor (= index of sphericity)

            'p-GG-corr': Greenhouse-Geisser corrected p-value

            'W-spher': Sphericity test statistic


            'p-spher': p-value of the sphericity test

            'sphericity': sphericity of the data (boolean)
                - variances between groups are equal. g1-g2 (+ all subjects) g1-g3. g2-g3. all equal?
        else:
            sum_sq: measure of total variation within data. how much data points differ from the mean
            df (deg of freedom): independ. values, values that are free to vary once we know
                            constraints (sample means)
            F-statistic:  model variance / residual variance. amount of variance due to the variable.
            P-value: is the variance noted (sum_sq, F-statistic) significant?
    """
    if anovaRM:
        # print(df.shape, df.head())
        anova_table = pg.rm_anova(dv='Quantity', within=['Chemical_Type', 'Gene'],
                  subject='Subject', data=df)
        anova_table.index = anova_table['Source']
    else:
        model = ols('Value ~ C(Gene) + C(Chemical_Type) + C(Gene):C(Chemical_Type)', data= df).fit()
        anova_table = sm.stats.anova_lm(model, typ=2)
        anova_table = anova_table.drop('Residual')
    return anova_table

ANOVA = {test_param: {} for test_param in comparisons_long.keys()}
for test_param, mutVcontrol_dict in comparisons_long.items():
    for mut, table in mutVcontrol_dict.items():
        if table['Chemical_Type'].nunique() > 1:
            ANOVA[test_param][mut]= calculate_ANOVA(test_param, table, anovaRM=ANOVARM).drop(columns='Source')

# ANOVA_dict['Mean Firing Rate (Hz)']['GPR37L_1']

In [50]:
ANOVA['Mean Firing Rate (Hz)']['GPR37L_1']

,SS,ddof1,ddof2,MS,F,p-unc,p-GG-corr,ng2,eps
Source,,,,,,,,,
Chemical_Type,6.2e+01,1,23,6.2e+01,1.0e+01,3.6e-03,3.6e-03,1.2e-01,1.0e+00
Gene,1.5e+00,1,23,1.5e+00,3.3e-01,5.7e-01,5.7e-01,3.2e-03,1.0e+00
Chemical_Type * Gene,1.4e+00,1,23,1.4e+00,4.1e-01,5.3e-01,5.3e-01,3.1e-03,1.0e+00


In [51]:
ANOVA['Area Under Cross-Correlation']['SLITRK5']

,SS,ddof1,ddof2,MS,F,p-unc,p-GG-corr,ng2,eps
Source,,,,,,,,,
Chemical_Type,8.4e+02,1,23,8.4e+02,2.0e+00,1.7e-01,1.7e-01,3.4e-02,1.0e+00
Gene,4.9e+01,1,23,4.9e+01,3.8e-01,5.4e-01,5.4e-01,2.0e-03,1.0e+00
Chemical_Type * Gene,8.0e-01,1,23,8.0e-01,5.2e-03,9.4e-01,9.4e-01,3.3e-05,1.0e+00


In [52]:
def create_multind_df(ANOVA_dict):
    pd.set_option('display.float_format', lambda x: '%.1e' % x)
    test_params = list(ANOVA_dict.keys())
    mutant_genes = list(ANOVA_dict[test_params[0]].keys())
    anova_variables = list(ANOVA_dict[test_params[0]][mutant_genes[0]].index)
    col_iterables = [mutant_genes, ["F-stat", "p-GG-corr","Eta^2 effect size-ng2"]]
    row_iterables = [test_params, anova_variables]
    
    rindex = pd.MultiIndex.from_product(row_iterables, names=["Test Parameter", "ANOVA variables"])
    cindex = pd.MultiIndex.from_product(col_iterables)
    
    df = pd.DataFrame(index=rindex, columns= cindex)
    for test_param, testparam_dict in ANOVA_dict.items():
        for mutant, anova_df in testparam_dict.items():
            for var in anova_variables:
                df.loc[(test_param, var), (mutant, 'F-stat')] = anova_df.loc[var, 'F']
                df.loc[(test_param, var), (mutant, 'p-GG-corr')] = anova_df.loc[var, 'p-GG-corr']
                df.loc[(test_param, var), (mutant, 'Eta^2 effect size-ng2')] = anova_df.loc[var, 'ng2']
                # df.loc[(test_param, 'C(Gene)'), (mutant, 'F-statistic')] = (anova_df['F']['C(Gene)'])
    return df

nf= create_multind_df(ANOVA)

nf

GPR37L_1            \
                                                        F-stat p-GG-corr   
Test Parameter                   ANOVA variables                           
Number of Spikes                 Chemical_Type         1.0e+01   3.6e-03   
                                 Gene                  3.3e-01   5.7e-01   
                                 Chemical_Type * Gene  4.1e-01   5.3e-01   
Mean Firing Rate (Hz)            Chemical_Type         1.0e+01   3.6e-03   
                                 Gene                  3.3e-01   5.7e-01   
...                                                        ...       ...   
Number of Covered Electrodes     Gene                  0.0e+00   1.0e+00   
                                 Chemical_Type * Gene -0.0e+00   1.0e+00   
Weighted Mean Resistance (kOhms) Chemical_Type         0.0e+00   1.0e+00   
                                 Gene                  8.5e+00   7.9e-03   
                                 Chemical_Type * Gene  0.0e+00   1.0e+00   

                                                                             \
                                                      Eta^2 effect size-ng2   
Test Parameter                   ANOVA variables                              
Number of Spikes                 Chemical_Type                      1.2e-01   
                                 Gene                               3.2e-03   
                                 Chemical_Type * Gene               3.1e-03   
Mean Firing Rate (Hz)            Chemical_Type                      1.2e-01   
                                 Gene                               3.2e-03   
...                                                                     ...   
Number of Covered Electrodes     Gene                               0.0e+00   
                                 Chemical_Type * Gene               0.0e+00   
Weighted Mean Resistance (kOhms) Chemical_Type                      0.0e+00   
                                 Gene                               4.3e-02   
                                 Chemical_Type * Gene               0.0e+00   

                                                      GPR37L_2            \
                                                        F-stat p-GG-corr   
Test Parameter                   ANOVA variables                           
Number of Spikes                 Chemical_Type         1.9e+01   2.2e-04   
                                 Gene                  1.4e+00   2.4e-01   
                                 Chemical_Type * Gene  1.8e+00   2.0e-01   
Mean Firing Rate (Hz)            Chemical_Type         1.9e+01   2.2e-04   
                                 Gene                  1.4e+00   2.4e-01   
...                                                        ...       ...   
Number of Covered Electrodes     Gene                  2.1e+00   1.6e-01   
                                 Chemical_Type * Gene  0.0e+00   1.0e+00   
Weighted Mean Resistance (kOhms) Chemical_Type         0.0e+00   1.0e+00   
                                 Gene                  1.2e+00   2.8e-01   
                                 Chemical_Type * Gene  2.3e-14   1.0e+00   

                                                                             \
                                                      Eta^2 effect size-ng2   
Test Parameter                   ANOVA variables                              
Number of Spikes                 Chemical_Type                      1.7e-01   
                                 Gene                               1.3e-02   
                                 Chemical_Type * Gene               1.7e-02   
Mean Firing Rate (Hz)            Chemical_Type                      1.7e-01   
                                 Gene                               1.3e-02   
...                                                                     ...   
Number of Covered Electrodes     Gene                               2.2e-02   
                           

In [58]:
## highlight rows when chemical and gene are significant.. 
def sigpval(p):
    lessthanp = p < 0.05
    return ['background-color: #CD5C5C' if v else '' for v in lessthanp]

psubset = [(col, 'p-GG-corr') for col in nf.columns.get_level_values(0).unique()]
msubset = [(col, 'Eta^2 effect size-ng2') for col in nf.columns.get_level_values(0).unique()]
fubset = [(col, 'F-stat') for col in nf.columns.get_level_values(0).unique()]
all = []
all.extend(psubset)
all.extend(msubset)
all.extend(fubset)

foldname = '-'.join(list(map(lambda x : x.split("_")[-1], folder_dict.keys())))
print(foldname)
if ANOVARM:
    SAVE_ANOVA_TABLE_FILE = f"{parent}/ANOVA_results/2wayANOVARM/{PERTURBATION}/{foldname}/{EXPERIMENT_TIME}/{plate_type}_{PERTURBATION}_ANOVARMresults.xlsx"
else:
    SAVE_ANOVA_TABLE_FILE = f"{parent}/ANOVA_results/2wayANOVA/{PERTURBATION}/{foldname}/{EXPERIMENT_TIME}/{plate_type}_{PERTURBATION}_ANOVAresults.xlsx"
ensure_dirs_exists(SAVE_ANOVA_TABLE_FILE)
nf.style.apply(sigpval, subset=psubset).to_excel(SAVE_ANOVA_TABLE_FILE)


basal-20uMCNO


In [54]:
nf.style.apply(sigpval, subset=psubset)